In [34]:
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint

from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings

# Own settings
from settings import *
workdir = os.getcwd()
moduledir, fcndir = defineDir(workdir)

## load TOOCAN
# from load_TOOCAN_DYAMOND_modif import load_TOOCAN_DYAMOND
from load_TOOCAN_DYAMOND_modif_BF import load_TOOCAN_DYAMOND

## current script object
thismodule = sys.modules[__name__]

Own modules available: ['conditionalstats', 'plot2D', 'plotCondPDFs', 'plot1D', 'conditionalstats_chunked']
Own modules available: ['fcns_load_DYAMOND_SAM', 'load_TOOCAN_DYAMOND_modif', 'load_TOOCAN_DYAMOND', 'load_TOOCAN_DYAMOND_modif_BF', 'casestudy', 'settings']


In [36]:
region = '205E_250E_0N_20N'
DATA_PATH = '/data/bfildier/DYAMOND_REGIONS'
DIR_SAM_2D = os.path.join(DATA_PATH,region,'SAM/2D')

# Relation table

# Case study per region

1 variable for this region weights 22G. Need to implement DistributionChunked. Do this in object CaseStudy.

In [45]:
from casestudy import *
from fcns_load_DYAMOND_SAM import *

In [46]:
region = 'tropics'
varid = 'Prec'
mask = 'all'

In [47]:
cs_tropics = CaseStudy(name='DYAMOND-SAM',
                  region=region,
                  rel_tab_dyam_seg=loadRelTable('DYAMOND_SEG'))

    Load all time-dependent distributions

In [48]:
! ls /home/bfildier/analyses/explore-multiscale-extremes/results/tropics/Prec/all

time_slices.tar.gz


In [52]:
! ls ../temp

In [53]:
cs_tropics.setSimulationSpecs(i_t_min = 832,
                              i_t_max = 1917,
                              lat_slice=slice(-30,30))

# where sliced distributions are stored
cs_tropics.setDirectories(dir_dist_sliced=os.path.join(DIR_OUT,region,varid,mask))

# load all distributions
cs_tropics.loadDistPrSliced()

    Compute Pr distribution over all times

In [54]:
# compute mean precip time series
cs_tropics.computePrMean()

# find "missing" time indices
cs_tropics.findTimeIndToIgnore()

In [55]:
# compute full distribution
cs_tropics.combinePrDistributions()

In [56]:
print(cs_tropics.dist_pr)

< DistributionChunked object:
 . bintype: invlogQ
 . nbins: 81
 . nbpd: 10
 . nppb: 4
 . nd: 8
 . fill_last_decade: False
 . name: pr, DYAMOND-SAM tropics, all times
 . size: 15238250496
 . vmin: -0.075976014
 . vmax: 125.83331
 . rank_edges: <class 'numpy.ndarray'>
 . ranks: <class 'numpy.ndarray'>
 . percentiles: <class 'numpy.ndarray'>
 . bins: <class 'numpy.ndarray'>
 . density: None
 . bin_locations_stored: [False False False ... False False False]
 . overwrite: False
 . dist_chunks: <class 'list'>
 . chunks_to_ignore: <class 'list'>
 . nchunks: 1086
 . size_chunks: <class 'list'>
 . vmin_chunks: <class 'list'>
 . vmax_chunks: <class 'list'>
 . bin_locations: {}
 >


In [57]:
print(cs_tropics)
print('time indices to ignore:')
print(cs_tropics.times_to_ignore)

< CaseStudy object:
 . name: DYAMOND-SAM
 . region: tropics
 . rel_tab_dyam_seg: <class 'pandas.core.frame.DataFrame'>
 . i_t_min: 832
 . i_t_max: 1917
 . range_t: range(832, 1918)
 . lat_slice: slice(-30, 30, None)
 . lon_slice: None
 . dir_dist_sliced: ../results/tropics/Prec/all
 . dict_dist_pr_sliced: <class 'dict'>
 . pr_mean: [0.08895927 0.09020749 0.09127147 ... 0.0871079  0.08649893 0.08537947]
 . times_to_ignore: <class 'numpy.ndarray'>
 . dist_pr: <class 'conditionalstats_chunked.DistributionChunked'>
 >
time indices to ignore:
[ 862  863  958  959 1054 1055 1150 1151 1246 1247 1342 1343 1438 1439
 1534 1535 1630 1631 1726 1727 1822 1823]


# Case study over warm pool

In [6]:
cs_warmpool = CaseStudy(name='DYAMOND-SAM',
                       region='Warm Pool',
                       rel_tab_dyam_seg=loadRelTable('DYAMOND_SEG'))

In [7]:
coords_warmpool = [130,165,0,20] # Western Pacific Warm Pool

In [9]:
cs_warmpool.setSimulationSpecs(i_t_min = 832,
                              i_t_max = 1917,
                              lat_slice=slice(coords_warmpool[2],coords_warmpool[3]),
                              lon_slice=slice(coords_warmpool[0],coords_warmpool[1]))